# QHO PINN Modulus Code

This code makes use of TensorFlow and Nvidia Modulus v21.06 (https://developer.nvidia.com/modulus). It is a demonstration for solving the 1D QHO system with a PINN.

In [ ]:
from sympy import Symbol, Function, Number, sin, I, pi, sqrt, re, im
from sympy.functions import exp

from modulus.pdes import PDES
from modulus.variables import Variables

import numpy as np

from modulus.solver import Solver
from modulus.dataset import TrainDomain, ValidationDomain
from modulus.data import Validation
from modulus.sympy_utils.geometry_1d import Line1D
from modulus.controller import ModulusController

import tensorflow as tf

Define the PDEs

In [ ]:
class QHOEq1D(PDES):

  name = 'qho_1d'

  def __init__(self, f=1.0):
    # coordinates
    x = Symbol('x')

    # time
    t = Symbol('t')

    # make input variables
    input_variables = {'x':x, 't':t}

    # make u,v function
    u = Function('u')(*input_variables)
    v = Function('v')(*input_variables)

    # frequency
    if type(f) is str:
      f = Function(f)(*input_variables)
    elif type(f) in [float, int]:
      f = Number(f)

    # set equations
    self.equations = Variables()
    self.equations['qho_1d_u'] = - u.diff(t) - 1/2 * (v.diff(x, 2)) + f**2/2 * (x**2) * v
    self.equations['qho_1d_v'] = - v.diff(t) + 1/2 * (u.diff(x, 2)) - f**2/2 * (x**2) * u

Set up the domains, initial conditions and boundary conditions.

In [ ]:
def get_ic(x):
    f=1.0
    phi_0 =(f / pi) ** (1. / 4.) * exp(-x * f * x / 2.0)
    phi_1 = phi_0 *  sqrt(f / 2.) * 2.0 * x
    psi = sqrt(1. / 2.) * (phi_0 + phi_1)
    return psi

# params for domain
L = float(np.pi)

# define geometry
geo = Line1D(-L, L)

# define sympy varaibles to parametize domain curves
t_symbol = Symbol('t')
time_range = {t_symbol: (0, 2*L)}

class QHOTrain(TrainDomain):
	def __init__(self, **config):
    super(QHOTrain, self).__init__()
    # sympy variables
    x = Symbol('x')

    #initial conditions
    IC = geo.interior_bc(outvar_sympy={'u': get_ic(x), 'v': 0.0},
                         bounds={x: (-L, L)},
                         batch_size_per_area=100,
                         lambda_sympy={'lambda_u': 1.0,
                                       'lambda_v': 1.0},
                         param_ranges={t_symbol: 0.0})
    self.add(IC, name="IC")


    #boundary conditions
    BC = geo.boundary_bc(outvar_sympy={'u': 0,'v': 0},
                         batch_size_per_area=100,
                         lambda_sympy={'lambda_u': 1.0,
                                       'lambda_v': 1.0},
                         param_ranges=time_range)
    self.add(BC, name="BC")

    # interior
    interior = geo.interior_bc(outvar_sympy={'qho_1d_u': 0,'qho_1d_v': 0},
                               bounds={x: (-L, L)},
                               batch_size_per_area=1000,
                               lambda_sympy={'lambda_qho_1d_u': 1.0,'lambda_qho_1d_v': 1.0},
                               param_ranges=time_range)
    self.add(interior, name="Interior")

Set up analytical solution and validation set.

In [ ]:
def get_analytical_solution(X,T):
    f=1.0
    phi_0 =(f / np.pi) ** (1. / 4.) * np.exp(-X * f * X / 2.0)
    phi_1 = phi_0 *  np.sqrt(f / 2.) * 2.0 * X
    psi = np.sqrt(1. / 2.) * (np.exp(-1j * f * T/2.0) * phi_0 + np.exp(-1j* 3 * f * T/2.0) * phi_1)

    return psi

class QHOVal(ValidationDomain):
  def __init__(self, **config):
    super(QHOVal, self).__init__()
    # make validation data
    deltaT = 0.01
    deltaX = 0.01
    x = np.arange(-L, L, deltaX)
    t = np.arange(0, 2*L, deltaT)
    X, T = np.meshgrid(x, t)
    X = np.expand_dims(X.flatten(), axis=-1)
    T = np.expand_dims(T.flatten(), axis=-1)
    
    psi_val = get_analytical_solution(X,T)
    u = np.real(psi_val)
    v = np.imag(psi_val)

    invar_numpy = {'x': X, 't': T}
    outvar_numpy = {'u': u, 'v': v}
    val = Validation.from_numpy(invar_numpy, outvar_numpy)
    self.add(val, name='Val')

Define and run the PINN.

In [ ]:
# Define neural network
class QHOSolver(Solver):
  train_domain = QHOTrain
  val_domain = QHOVal

  def __init__(self, **config):
    super(QHOSolver, self).__init__(**config)

    #self.equations = QHOEq1D(f=1.0).make_node()
    self.equations = (QHOEq1D(f=1.0).make_node())
    qho_net = self.arch.make_node(name='qho_net',
                                   inputs=['x', 't'],
                                   outputs=['u','v'])
    self.nets = [qho_net]

  @classmethod
  def update_defaults(cls, defaults):
    defaults.update({
        'network_dir': './checkpoints/network_checkpoint_qho_1d_full_single_func_3000',
        'max_steps': 3000,
        'decay_steps': 1000,
        })

if __name__ == '__main__':
    #tf.logging.set_verbosity(tf.logging.ERROR)
    ctr = ModulusController(QHOSolver)
    ctr.run()